In [3]:
import pandas as pd
df = pd.read_csv("D:/Courses/Codebasics Python/Python Projects and Exercises/ML/9_decision_tree/Exercise/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [47]:
df['Age'].median()

28.0

### In this project, I will assume the 4 factors Pclass, Sex, Age and Fare to have had some sort of relationship with Survival.
**So, those 4 will be my independent or X variables, and Survival will be my target or Y variable that I will try to predict**

In [7]:
inputs = df[['Pclass', 'Sex', 'Age', 'Fare']]

In [9]:
target = df['Survived']

**Encoding the Categorical Variable, Sex**

In [11]:
from sklearn.preprocessing import LabelEncoder

In [24]:
le = LabelEncoder()

In [65]:
inputs['sex_n'] = le.fit_transform(inputs['Sex'])

KeyError: 'Sex'

**Adding This Encoded Column to the Original Dataframe**

In [30]:
sex_n = pd.DataFrame(sex_n, columns=['sex_n'])
inputs_n = pd.concat([inputs, sex_n],axis='columns')

**The above way would have been the normal way** ***BUT*** **refer back to Line[28]. The reason I wrote inputs['sex_n'] instead of only 'sex_n' is so that the variable is not only created but also added as a column to the inputs DataFrame.** 

In [31]:
inputs

,Pclass,Sex,Age,Fare,sex_n
0,3,male,22.0,7.2500,1
1,1,female,38.0,71.2833,0
2,3,female,26.0,7.9250,0
3,1,female,35.0,53.1000,0
4,3,male,35.0,8.0500,1
...,...,...,...,...,...
886,2,male,27.0,13.0000,1
887,1,female,19.0,30.0000,0
888,3,female,NaN,23.4500,0
889,1,male,26.0,30.0000,1


**Now, we have to drop the String Column**

inputs = inputs.drop(['Sex'], axis = 'columns') 
    *bcs running twice showing error since 'Sex' column already has been removed once.*

In [36]:
inputs

,Pclass,Age,Fare,sex_n
0,3,22.0,7.2500,1
1,1,38.0,71.2833,0
2,3,26.0,7.9250,0
3,1,35.0,53.1000,0
4,3,35.0,8.0500,1
...,...,...,...,...
886,2,27.0,13.0000,1
887,1,19.0,30.0000,0
888,3,NaN,23.4500,0
889,1,26.0,30.0000,1


**Splitting Data into Training and Validation Dataset**

In [37]:
from sklearn.model_selection import train_test_split

In [57]:
x_train, x_test, y_train, y_test = train_test_split(inputs, target, train_size = 0.8)

**Model Calling | Decision Tree**

In [58]:
from sklearn import tree

In [59]:
model = tree.DecisionTreeClassifier()

**Fitting Model**

In [42]:
model.fit(x_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

### Ok so we ran into our first problem of dirty data. We have to clean the dataset because there are missing values here.
### Let's see how.

In [49]:
inputs.isna().sum()

Pclass      0
Age       177
Fare        0
sex_n       0
dtype: int64

**Great! So we see that, thanfully, the column data are clean in general. Only the age data has instances of missing data**

**We can clean it by many methods: dropping NA, filling NA with an appropriate value, interpolating, etc.**

**In this case, I will take the mean of the ages, as this seems reasonable**

In [61]:
from numpy import mean
inputs = inputs.fillna(mean(inputs.Age))

In [62]:
inputs.isna().sum()

Pclass    0
Age       0
Fare      0
sex_n     0
dtype: int64

**There! Now the missing values are gone. We have successfully cleaned the data**

**Now it's ready for model fitting. I will now fit the data into my Decision Tree Model**

In [63]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

**Model Accuracy & Score**

In [64]:
model.score(x_test, y_test)

0.776536312849162

### Let's do Predictions!

**But wait, we have to know what inputs to give to explain to the Model what we mean**

**Let's find the relevant indexes for our only categorical data, in this project, Sex i.e. whther male or female**

In [70]:
inputs_n = pd.concat([inputs, df.Sex],axis='columns')

le = LabelEncoder()

for col in inputs_n.Sex:
    if inputs_n['Sex'].dtype == 'object':
        le.fit(inputs_n['Sex'])
        print('Sex:', dict(zip(le.classes_, le.transform(le.classes_))))
        break

Sex: {'female': 0, 'male': 1}


**Alright, back to Predictions!**

In [75]:
model.predict([[2, 14, 30.0708, 0]])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)